In [1]:
import pandas as pd
import os
import glob
import numpy as np

# Smartsheets 

In [2]:
path1 = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*"
file_type1 = ".xlsx"
load1 = glob.glob(path1 + file_type1)
df = pd.read_excel(max(load1, key=os.path.getctime), parse_dates=True, keep_default_na=False)

In [3]:
path2 = r"C:\Users\jacque.trahan\Downloads\South Central Division 2021 Debris Removal Tracker*"
load2 = glob.glob(path2 + file_type1)
df2 = pd.read_excel(max(load2, key=os.path.getctime), parse_dates=True, keep_default_na=False)

In [4]:
df_both = pd.concat([df, df2])

In [5]:
# df_both.info(verbose=True)

In [6]:
check = df_both[['APN_ROW Segment', 'Structural Status', 'Street #', 'Street Name','County', 'Debris Crew','ROE Type',
                 'Haz Tree Status','Tree Felling Start', 'Tree Felling Complete','Trees Completion Check',
                 'Soil Samples Approved',
                 'Erosion Control Complete','Erosion Control Waiver',]]

In [7]:
check.to_excel('Check FSW and Debris Crew.xlsx')

In [8]:
FSO = df_both.copy()
FSO = FSO[['APN_ROW Segment', 'Structural Status', 'Street #', 'Street Name','County',
                  'Haz Tree Status',"Tree Removal Complete", # should say TFSW complete willneed to check this
                  'Soil Samples Approved',
                  'Erosion Control Complete', 'Erosion Control Waiver',
                  "USFS Parcel?", "Needs Milk Run or Phase 1 Revisit",
                   "811 Called (A&M)", "811 Called (DDHTR)",
                   "Vehicle(s)", 'Number of Vehicles Removed',
                   "ROE Date",
                   'FSO Complete', 'FSO Sent to County','Debris Crew',
           'Number of Trees','Tree Removal Start', 'Tree Removal Complete',
           'Tree Felling Start', 'Tree Felling Complete',
           'Number of Hazard Trees Felled','Trees Completion Check', 'ROE Type', 'Tree Crew', 'Trees Completion Check',
           'Total Number of Vehicles',
           'Hold', 'Hold Reason'
       ]]
FSO["ROE Date"] = FSO["ROE Date"].fillna("")
FSO.set_index('APN_ROW Segment', inplace=True)
FSO.to_excel("Smartsheets data.xlsx")
FSO.reset_index(inplace=True)

# RTR Tree

In [9]:
path2 = r"C:\Users\jacque.trahan\Downloads\aDataManagerTicketExport*"
file_type2 = ".xlsx"
load2 = glob.glob(path2 + file_type2)
df2 = pd.read_excel(max(load2, key=os.path.getctime), parse_dates=True)

In [10]:
RTR = df2.copy()
RTR = RTR[['Zone Name', 'Service Code', 'Unit Count', 'Is Void', 'GIS Zone1','GIS Zone2','Start Time', 'End Time',"Addr No", "Addr St", "Ticket Number"]]
RTR = RTR.copy()
RTR['Zone Name'] = RTR['Zone Name'].str[:3] + "-" + RTR['Zone Name'].str[3:6] + "-" + RTR['Zone Name'].str[6:]
RTR['Service Code'] = RTR['Service Code'].str.strip()
RTR = RTR[(RTR['Is Void'] == False) & (RTR['Service Code'].isin(['12A', '12B', '76', '55A','55B'])) & (RTR['GIS Zone1'].isin(['CENTRAL DIVISION']))] 

In [11]:
RTR.set_index('Zone Name', inplace = True)
RTR.to_excel('RTR Data for Trees.xlsx')

# Nightly Tree Email for Ready for TFSW

In [12]:
# rdy_TFSW = df.copy()
# rdy_TFSW = rdy_TFSW[['APN_ROW Segment', 'Tree Crew', 'Street #', 'Street Name','County','Tree Removal Start', 'Tree Removal Complete'
#        ]]
# rdy_TFSW = rdy_TFSW[rdy_TFSW['Tree Crew'].isin(['Ready for TFSW'])]
# rdy_TFSW.set_index('APN_ROW Segment')
# rdy_TFSW.to_excel("Ready for TFSW-Central Dixie 2021.xlsx")

# RTR Data For Erosion

In [13]:
RTR2 = df2.copy()
RTR2 = RTR2[['Zone Name', 'Service Code', 'Unit Count', 'Is Void', 'GIS Zone1','GIS Zone2', 'Ticket Date',
             "Addr No", "Addr St", "Ticket Number"]]
RTR2 = RTR2.copy()
RTR2['Zone Name'] = RTR2['Zone Name'].str[:3] + "-" + RTR2['Zone Name'].str[3:6] + "-" + RTR2['Zone Name'].str[6:]
RTR2['Service Code'] = RTR2['Service Code'].str.strip()
RTR_erosion = RTR2[(RTR2['Is Void'] == False) & (RTR2['Service Code'] == '73A')] 
RTR_erosion =  RTR_erosion[RTR_erosion['GIS Zone1'].isin(['CENTRAL DIVISION'])]
RTR_erosion.set_index('Zone Name', inplace=True)
RTR_erosion.to_excel('RTR Data For Erosion.xlsx')

# RTR_erosion = RTR_erosion.add_suffix('_RTR')
# RTR_erosion.reset_index(inplace=True)

In [14]:
# final = FSO.merge(RTR_erosion,
#                      left_on='APN_ROW Segment',
#                      right_on='Zone Name',
#                      how='outer')


In [15]:
# final.columns

In [16]:

# final.set_index('APN_ROW Segment', inplace=True)
# final.to_excel('Smart sheets and Erosion files side by side.xlsx')